# Generamos Foto Previsión desde Fotos Histórico

_Importación de librerias_

In [2]:
import pandas as pd
import tfm_helpers as tfm

pd.set_option("display.max_columns", None)

Procesamos los ficheros para generar previsiones agrupadas por año

In [3]:
# # years_to_process = ["2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019"]
# years_to_process = ["2005"]
# for year in years_to_process:
#     files = tfm.getListOfFiles(tfm.getPathToProcessedSnapshotReservasFolder("parquet", year))
#     current_snapshot_data = pd.DataFrame()
#     result_ocupacion_data = pd.DataFrame()
#     for file in files:
#         current_snapshot_data = pd.read_parquet(file)
#         filtro_reservas_activas = ((current_snapshot_data["Estado reserva"] == "Reserva") | (current_snapshot_data["Estado reserva"] == "Entrada"))
#         current_snapshot_data = current_snapshot_data[filtro_reservas_activas]
#         current_snapshot_data["Fecha prevision"] = [pd.date_range(entrada, salida, freq="d", closed="left") for entrada, salida in 
#                                                     zip(pd.to_datetime(current_snapshot_data["Fecha entrada"]), 
#                                                         pd.to_datetime(current_snapshot_data["Fecha salida"]))]
#         current_snapshot_data = current_snapshot_data.explode("Fecha prevision")
#         result_ocupacion_data = pd.concat([result_ocupacion_data, current_snapshot_data]) 

#     result_ocupacion_data.set_index(["Hotel", "Fecha foto", "Fecha prevision"], inplace=True)
#     result_ocupacion_data.sort_index(inplace=True)
#     result_ocupacion_data.to_parquet(tfm.getPathToProcessedSnapshotOcupacion("parquet", year), index=True)
#     result_ocupacion_data.to_csv(tfm.getPathToProcessedSnapshotOcupacion("csv", year), index=True, sep="|", escapechar="~", encoding="cp1252")

In [6]:
# years_to_process = ["2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019"]
# years_to_process = ["2014", "2015", "2016", "2017", "2018", "2019"]
years_to_process = ["2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019"]
for year in years_to_process:
    files = tfm.getListOfFiles(tfm.getPathToProcessedSnapshotReservasFolder("parquet", year))
    result_ocupacion_data = pd.DataFrame()
    for file in files:
        current_snapshot_data = pd.read_parquet(file, columns=["Hotel", "Fecha venta", "Fecha entrada", "Fecha salida", "Noches", "Cantidad habitaciones", "Estado reserva", "Fecha Borrado", "Días Anticipación", "Fecha foto"])
        current_snapshot_data.reset_index(inplace=True)
        current_snapshot_data["Hotel"] = current_snapshot_data.Hotel.astype("category")
        current_snapshot_data["Estado reserva"] = current_snapshot_data["Estado reserva"].astype("category")        
        
        filtro_reservas_activas = ((current_snapshot_data["Estado reserva"] == "Reserva") | (current_snapshot_data["Estado reserva"] == "Entrada"))
        current_snapshot_data = current_snapshot_data[filtro_reservas_activas]

        fecha_foto = current_snapshot_data["Fecha foto"].min()
        current_snapshot_data["Fecha inicio"] = current_snapshot_data["Fecha entrada"].apply(lambda x : x if x > fecha_foto else fecha_foto)
        
        current_snapshot_data["Fecha prevision"] = [pd.date_range(entrada, salida, freq="d", closed="left") for entrada, salida in 
                                                    zip(pd.to_datetime(current_snapshot_data["Fecha inicio"]), 
                                                        pd.to_datetime(current_snapshot_data["Fecha salida"]))]
        current_snapshot_data.drop(columns=["Fecha inicio"], inplace=True)   
        current_snapshot_data = current_snapshot_data.explode("Fecha prevision")
        result_ocupacion_data = pd.concat([result_ocupacion_data, current_snapshot_data]) 

    # result_ocupacion_data.set_index(["Hotel", "Fecha foto", "Fecha prevision"], inplace=True)
    # result_ocupacion_data.sort_index(inplace=True)
    result_ocupacion_data.to_parquet(tfm.getPathToProcessedSnapshotOcupacion("parquet", year), index=False)
    # result_ocupacion_data.to_csv(tfm.getPathToProcessedSnapshotOcupacion("csv", year), index=True, sep="|", escapechar="~", encoding="cp1252")